In [1]:
import numpy as np
import seaborn as sns
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm
from small2DNet import small2DNet
from util import add_color, colorize, colorize_gaussian, calculate_correct_loss
from colorMNist import colorMNist
import random
import colorsys
import pickle
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import wandb

In [2]:
wandb.init(project="my-test-project", entity="ntahur")

wandb: Currently logged in as: ntahur (use `wandb login --relogin` to force relogin)


In [9]:
wandb.finish()

Cavgsim-0,██▆▄▃▃▂▂▁▁
Cavgsim-1,▁▂▄███▇▇▆▆
Cavgsim-2,▁▁▂▆████▇▇
Cavgsim-3,▁▁▂▇▇█████
Cavgsim-4,███▆▄▃▂▂▁▁
Cavgsim-5,▁▁▂▃▅▅▅▆▇█
Cavgsim-6,▃▁▁▂▄▄▅▆▆█
Cavgsim-7,█▇▃▁▃▄▄▄▄▄
Uavgsim-0,██▇▆▂▁▁▁▂▂
Uavgsim-1,▁▁▁▂▆██▇▆▆
Uavgsim-2,▁▁▁▁▅█████


In [ ]:
wandb.config = {
  "learning_rate": 0.01,
  "epochs": 10,
  "batch_size": 32
}

In [ ]:
wandb.log({"loss": 12})

In [ ]:
wandb.log({"loss": 12})

In [6]:
# Load data from pickle file
cmnist_train, cmnist_val, cmnist_test = pickle.load(open("custom_datasets/cmnist_colorless.pkl", "rb"))
print(len(cmnist_train), len(cmnist_val), len(cmnist_train) + len(cmnist_val))

# Create datasets
train_dataset = colorMNist(cmnist_train)
val_dataset = colorMNist(cmnist_val)
test_dataset = colorMNist(cmnist_test)

# Dataloaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                               shuffle=True, num_workers = 0)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32,
                                               shuffle=True, num_workers = 0)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32,
                                               shuffle=False, num_workers = 0)

4000 1000 5000


In [ ]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='datasets/cifar10', train=True, download=False, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='datasets/cifar10', train=False, download=False, transform=transform)

# Create the 80/20 train/val split
train_split = int(0.8 * len(trainset))
# Train dataset
train_dataset = torch.utils.data.Subset(trainset, [i for i in range(train_split)])
# Validation dataset
val_dataset = torch.utils.data.Subset(trainset, [i for i in range(train_split, len(trainset))])
print(len(train_dataset), len(val_dataset), len(train_dataset) + len(val_dataset))

train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [7]:
# Layers of the model
model_layers = [8, 8, "M", 16,"M"]
# model_layers = [24, 24, "M", 48,"M"]
# Set seed
torch.manual_seed(12)
# Create model
linear_neurons = 512
model = small2DNet(model_layers, 16, linear_neurons)
# Load file and save file
lfile = "3Deterministic2D"
sfile = "Deterministic2D"
# Load model
# model.load_state_dict(torch.load('model_saves/Triple2D/'+ lfile + '.pth'))
# Put model on gpu
model.cuda()
# Loss function
loss_fn = torch.nn.CrossEntropyLoss()
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
# Freeze weights
children = [x for x in model.children()]
for x in children[0]:
    for param in x.parameters():
        param.requires_grad = False

In [8]:
# Number of epochs to train
epochs = 10

# Placeholder variables to put training and validation accuracies and losses per epoch
train_accuracies = []
train_losses = []
val_accuracies = []
val_losses = []

for epoch in range(epochs):
    print("Epoch", epoch + 1, "/", epochs)
    
    # Update learning rate
    # if (epoch + 1) % 3 == 0:
    #     for g in optimizer.param_groups:
    #         g['lr'] /= 10
    
    # if (epoch + 1) % 5 == 0:
    children = [x for x in model.children()]

    for i, x in enumerate(children[0][0].weight):
        x0 = x[0][0].cpu().detach().numpy().flatten().reshape(1, -1)
        x1 = x[0][1].cpu().detach().numpy().flatten().reshape(1, -1)
        x2 = x[0][2].cpu().detach().numpy().flatten().reshape(1, -1)
        # wandb.log({"Usim01-"+str(i): cosine_similarity(x0, x1)[0][0]})
        # wandb.log({"Usim02-"+str(i): cosine_similarity(x0, x2)[0][0]})
        # wandb.log({"Usim12-"+str(i): cosine_similarity(x1, x2)[0][0]})
        wandb.log({"Cavgsim-"+str(i): (cosine_similarity(x0, x1)[0][0] + cosine_similarity(x0, x2)[0][0] + cosine_similarity(x1, x2)[0][0]) / 3})
    
    # Put model on training mode
    model.train()
    train_total_correct = 0
    train_total_loss = []
    
    for (images, labels) in tqdm(train_dataloader):
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Calculate number correct and loss in batch
        correct, loss = calculate_correct_loss(model, loss_fn, images, labels)
        
        # Backpropagation
        loss.backward()
        # Step function
        optimizer.step()
        
        # Update amount correct and loss with current batch
        train_total_correct += correct
        train_total_loss.append(loss.item())
        
    # Append epoch accuracy and loss
    train_accuracies.append(train_total_correct / len(train_dataset))
    train_losses.append(sum(train_total_loss) / len(train_total_loss))
    
    # Put model on evaluation mode
    model.eval()
    val_total_correct = 0
    val_total_loss = []
    
    # Without gradient calculation
    with torch.no_grad():
        for (images, labels) in tqdm(val_dataloader):
        
            # Calculate number correct and loss in batch
            correct, loss = calculate_correct_loss(model, loss_fn, images, labels)

            # Update amount correct and loss with current batch
            val_total_correct += correct
            val_total_loss.append(loss.item())

    # Append epoch accuracy and loss
    val_accuracies.append(val_total_correct / len(val_dataset))
    val_losses.append(sum(val_total_loss) / len(val_total_loss))
    

# Print accuracies and losses per epoch
for i in range(epochs):
    print("Epoch", i + 1)
    print("Train acc and loss\t", train_accuracies[i], "\t", train_losses[i])
    print("Val acc and loss\t", val_accuracies[i], "\t", val_losses[i])

Epoch 1 / 10


100%|██████████| 32/32 [00:00<00:00, 813.19it/s]


Epoch 2 / 10


100%|██████████| 32/32 [00:00<00:00, 859.87it/s]


Epoch 3 / 10


...

Epoch 9 / 10


100%|██████████| 32/32 [00:00<00:00, 654.77it/s]


Epoch 10 / 10


100%|██████████| 32/32 [00:00<00:00, 509.30it/s]

Epoch 1
Train acc and loss	 0.10675 	 2.3007213401794435
Val acc and loss	 0.119 	 2.293550565838814
Epoch 2
Train acc and loss	 0.20675 	 2.2805986289978026
Val acc and loss	 0.446 	 2.2195304483175278
Epoch 3
Train acc and loss	 0.5285 	 1.4642793869972228
Val acc and loss	 0.799 	 0.654135218821466
Epoch 4
Train acc and loss	 0.78025 	 0.6742028083801269
Val acc and loss	 0.858 	 0.42327648866921663
Epoch 5
Train acc and loss	 0.85675 	 0.4434449883103371
Val acc and loss	 0.889 	 0.3223112856503576
Epoch 6
Train acc and loss	 0.90075 	 0.32634836953878404
Val acc and loss	 0.933 	 0.2273154665599577
Epoch 7
Train acc and loss	 0.91675 	 0.26654954904317857
Val acc and loss	 0.921 	 0.2821725237299688
Epoch 8
Train acc and loss	 0.921 	 0.23926885211467744
Val acc and loss	 0.934 	 0.20293090969789773
Epoch 9
Train acc and loss	 0.935 	 0.20947843539714814
Val acc and loss	 0.948 	 0.16851679602405056
Epoch 10
Train acc and loss	 0.94575 	 0.17777234359085559
Val acc and loss	 0.953

In [ ]:
with open('fixed_trainvalAccs/' + sfile + '.txt', 'w') as f:
    for i in range(epochs):
        f.write("Epoch " + str(i + 1) + "\n")
        f.write("Train acc and loss\t" + str(train_accuracies[i]) + "\t" + str(train_losses[i]) + "\n")
        f.write("Val acc and loss\t" + str(val_accuracies[i]) + "\t" + str(val_losses[i]) + "\n")

In [ ]:
# Save the model
torch.save(model.state_dict(), 'fixed_model_saves/2D/' + sfile + '.pth')

In [ ]:
# Layers of the model
model_layers = [8, 8, "M", 16,"M"]
# model_layers = [24, 24, "M", 48,"M"]
# Create model
model = small2DNet(model_layers, 16, linear_neurons)
# Load model
model.load_state_dict(torch.load('fixed_model_saves/2D/'+ sfile + '.pth'))
# Put model on gpu
model.cuda()
hi = 5

In [ ]:
# Testing
    
wrong_dict = {}
right_dict = {}

for i in range(10):
    wrong_dict[i] = {}
    for j in range(10):
        wrong_dict[i][j] = 0
    right_dict[i] = 0
    
for it in range(1):
    # Total and amount correct
    test_correct = 0
    test_total = 0

    # File to write predicted labels
    # with open("predicted_labels.txt", "w") as f:
    #     f.write("\n")


    # Put the model in evaluation mode
    model.eval()

    # Without gradient calculation
    with torch.no_grad():
        for (images, labels) in tqdm(test_dataloader):

            # Put images
            images = images.cuda()

            # Predicted labels
            preds = model(images)

            # Top predictions per image
            _, top_preds = torch.max(preds, 1)

            # Predictions and images back on cpu
            top_preds = top_preds.cpu()
            images = images.cpu()
            
            # Check the predicted
            for i in range(len(labels)):
                if top_preds[i].item() == labels[i].item():
                    right_dict[top_preds[i].item()] += 1
                else:
                    wrong_dict[labels[i].item()][top_preds[i].item()] += 1

            # Amount of correct predictions
            predictions = [top_preds[i].item() == labels[i].item() for i in range(len(labels))]
            correct = np.sum(predictions)

    #         # Show batch images
    #         fig, axs = plt.subplots(4,8, figsize=(28, 28), facecolor='w', edgecolor='k')
    #         fig.subplots_adjust(hspace = .5, wspace=.001)
    #         axs = axs.ravel()
    #         for i in range(len(images)):
    #             axs[i].imshow(images[i].permute(1, 2, 0))

    #         break

            # Update total correct and total images
            test_correct += correct
            test_total += len(images)


    print("Correct", test_correct, "/", test_total, "Accuracy:", test_correct / test_total)

In [ ]:
for x in right_dict:
    print(x, ":", right_dict[x])

In [ ]:
hm = []
for i, x in enumerate(wrong_dict):
    hm.append([])
    for y in wrong_dict[x]:
        hm[i].append(wrong_dict[x][y])
    # print(x, ":", wrong_dict[x])
print(hm)

In [ ]:
sns.heatmap(hm, cmap="Blues")